<a href="https://colab.research.google.com/github/MHDBST/BERT_examples/blob/master/MyBert_document_partoken_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf
import pandas as pd
!pip install bert-tensorflow

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)



from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

('TPU address is', 'grpc://10.22.212.154:8470')


W1114 06:09:43.032152 140638232864640 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 6962112540401321833),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 519686666092638636),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 1444464901182362526),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4004541168969870320),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 5493426303070595074),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 9548215211541479504),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 13627310804106998582),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 3875737990770768375),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 16447755211679079350),
 _DeviceAttributes(/job:tpu_worke

In [2]:
import sys

# !test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
# if not 'bert_repo' in sys.path:
#   sys.path += ['bert_repo']

# import python modules defined by BERT
from bert import modeling
from bert import run_classifier_with_tfhub 
import tensorflow_hub as hub

W1114 06:09:44.662709 140638232864640 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/bert/optimization.py:87: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [3]:
BUCKET = 'bert_example' #@param {type:"string"}
TASK = 'MASKED_DOCUMENT'
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/aug19_models/models/docInfo/smallBERT-docLevel-seq512/true_label'.format(BUCKET,TASK)
# OUTPUT_DIR = 'gs://{}/bert-tfhub/models/nontrainable/smallBERT-docLevel-seq512'.format(BUCKET)

tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL


***** Model output directory: gs://bert_example/aug19_models/models/docInfo/smallBERT-docLevel-seq512/true_label *****


In [0]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

# # BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1" ##Small Bert
# # BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-24_H-1024_A-16/1" ##Big Bert
# def create_tokenizer_from_hub_module():
#   """Get the vocab file and casing info from the Hub module."""
#   with tf.Graph().as_default():
#     bert_module = hub.Module(BERT_MODEL_HUB)
#     tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
#     with tf.Session() as sess:
#       vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
#                                             tokenization_info["do_lower_case"]])
      
#     f_in = tf.gfile.GFile(vocab_file)
#     lines = f_in.readlines()

#     f_out = tf.gfile.GFile(vocab_file,'w')

#     lines[1] = '[TGT]\n'
#     lines[2] = '[tgt]\n'
#     lines[3] = 'TGT\n'
#     lines[4] = 'tgt\n'
#     for line in lines:
#       f_out.write(line)
#     f_out.close()

      
    
#     print('vocab_file>>' , vocab_file)
    
      
#   return bert.tokenization.FullTokenizer(
#       vocab_file=vocab_file, do_lower_case=do_lower_case)

# tokenizer = create_tokenizer_from_hub_module()

path = 'gs://bert_example/bert/uncased_L-12_H-768_A-12/vocab_tgt.txt'
f_in = tf.gfile.GFile('gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt')
f_out = tf.gfile.GFile(path,'w')
lines = f_in.readlines()


lines[1] = 'tgt\n'
for line in lines:
  f_out.write(line)
f_out.close()

VOCAB_FILE = os.path.join('gs://bert_example/bert/%s'%str(BERT_MODEL), 'vocab_tgt.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)

In [0]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 10.0  ## Activate if ** is ACTIVATED
MAX_SEQ_LENGTH = 512
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 20
SAVE_SUMMARY_STEPS = 10
trainable=True
keep_checkpoint_max = 15


In [6]:
from tensorflow import keras
import os
import re
import pandas as pd

# directory = '/content/alldata_3Dec_7Dec_PS_reindex_train_v3.csv'
# data_train = pd.read_csv('/content/alldata_3Dec_7Dec_PS_reindex_train_v3.csv', encoding='latin-1')
# data_dev = pd.read_csv('/content/alldata_3Dec_7Dec_PS_reindex_dev_v3.csv', encoding='latin-1')
# data_test = pd.read_csv('/content/alldata_3Dec_7Dec_PS_reindex_random_test_v3.csv', encoding='latin-1')
# data_test_fixed = pd.read_csv('/content/alldata_3Dec_7Dec_PS_reindex_fixed_test_v3.csv', encoding='latin-1')



# train_file = open('/content/train_v5.csv')
# train_df = pd.read_csv(train_file)[:-1]

# dev_file = open('/content/dev_v5.csv')
# dev_df = pd.read_csv(dev_file)[:-1]

# test_random_file = open('/content/random_test_v5.csv')
# test_random_df = pd.read_csv(test_random_file)[:-1]

# test_fixed_file = open('/content/fixed_test_v5.csv')
# test_fixed_df = pd.read_csv(test_fixed_file)[:-1]

data_pref = 'gs://bert_example/data_aug19/all_data_combined_shuffled_3Dec_7Dec_aug19_%s.csv'
train_df = pd.read_csv(tf.gfile.GFile(data_pref % str('train')), encoding='latin-1')
# dev_df = pd.read_csv(tf.gfile.GFile(data_pref % str('dev')), encoding='latin-1')
dev_df = pd.read_csv(tf.gfile.GFile('gs://bert_example/data_aug19/abstractive_pgn_summary_dev_comma.csv'), encoding='latin-1')
test_random_df= pd.read_csv(tf.gfile.GFile(data_pref % str('random_test')), encoding='latin-1')
test_fixed_df= pd.read_csv(tf.gfile.GFile(data_pref % str('fixed_test')), encoding='latin-1')


def fix_doc_tgt(doc,index,df):
  new_doc = doc.replace('.[TGT]',' .\n[TGT] ')
  doc_length = len(new_doc.split('\n'))
  if doc_length == 16 or not pd.notnull(df['Paragraph%s'%str(doc_length)].iloc[index]):
    return new_doc

  new_doc = new_doc.replace('?[TGT]',' ?\n[TGT] ')
  doc_length = len(new_doc.split('\n'))
  if doc_length == 16 or  not pd.notnull(df['Paragraph%s'%str(doc_length)].iloc[index]):
    return new_doc
  new_doc = new_doc.replace( '[TGT][TGT]', '[TGT] \n [TGT]' )
  doc_length = len(new_doc.split('\n'))
  if doc_length == 16 or  not pd.notnull(df['Paragraph%s'%str(doc_length)].iloc[index]):
    return new_doc

  new_doc = new_doc.replace( '. USA TODAY', '. USA TODAY Sports \n' )
  doc_length = len(new_doc.split('\n'))
  if doc_length == 16 or  not pd.notnull(df['Paragraph%s'%str(doc_length)].iloc[index]):
    return new_doc
  
  new_doc = new_doc.replace('![TGT]','!\n[TGT]' )
  doc_length = len(new_doc.split('\n'))
  return new_doc

## if pl is true, append  paragraph leve data and label to the output
## if dl is true, append document leve data and label to the output
#### Both pl and dl can not be false
def load_paragraphs_documents(df,info,txt):
    labels = []
    texts = []
    num_doc = 0
    index = -1
    
    for doc in df['MASKED_DOCUMENT']:
      
      new_docs = []
      index += 1
      docs = doc.split('\n')
      doc_length = len(docs)

      if pd.isnull(df['Paragraph0'].iloc[index]):
      # add documents with no paragraph labels as one document and its label to the input data dataframe
        doc_label = df['TRUE_SENTIMENT'].iloc[index]
        labels.append(df['TRUE_SENTIMENT'].iloc[index])
        new_doc = doc.replace('[TGT]','tgt')
        
        if info and txt:
            new_docs.append(doc_label +' ' + new_doc)
        elif txt:
            new_docs.append(new_doc)
        elif info:
            new_docs.append(doc_label)
        else:
            print('no information requested')
        texts.append('\n'.join(new_docs))
        num_doc +=1
        continue
      try:
        if doc_length != 16 and pd.notnull(df['Paragraph%s'%str(doc_length)].iloc[index]):
          doc = fix_doc_tgt(doc,index,df)
          docs = doc.split('\n')
          doc_length = len(docs)
          if  doc_length != 16 and pd.notnull(df['Paragraph%s'%str(doc_length)].iloc[index]):
            print('error on document %d'% df['DOCUMENT_INDEX'].iloc[index])
            print('document length is %s'%str(doc_length))
            continue
        if pd.isnull(df['Paragraph%s'%str(doc_length-1)].iloc[index]):
            print('document %d has %d paragraph labels and %d paragraphs'%(index,doc_length-1,doc_length))
            continue    
            
      except Exception as e:
        print('error occured: %s'%str(e))
        print('this document has %d paragraphs %d' %(doc_length,df['DOCUMENT_INDEX'].iloc[index]))
      # doc = doc.replace('[TGT]','tgt ')
      # docs = doc.split('\n')
      # doc_length = len(docs)

      for i in range(doc_length):
          
          label_i = df['Paragraph%d'%i].iloc[index]
#           print(label_i)
#           print(index)
          if info and txt:
            new_docs.append(label_i +' ' + docs[i])
          elif txt:
            new_docs.append(docs[i])
          elif info:
            new_docs.append(label_i)
          else:
            print('no information requested')
      labels.append(df['TRUE_SENTIMENT'].iloc[index])
      texts.append('\n'.join(new_docs))   
      

    print('number of one-paragraph docs: %d'%num_doc)
    return(texts,labels)
  


# Load all files from a directory in a DataFrame.
def load_file(df,info=False,txt=True):
  data = {}
  (texts,labels ) = load_paragraphs_documents(df,info=info,txt=txt)
  
  
  data["sentence"] = texts
  data["sentiment"] =labels
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(df,info=False,txt=True):
  df_new = load_file(df,info=info,txt=txt)
  pos_df = df_new[df_new['sentiment'] == 'Positive']
  neg_df = df_new[df_new['sentiment'] == 'Negative']
  neu_df = df_new[df_new['sentiment'] == 'Neutral']
  pos_df["polarity"] = 1
  neg_df["polarity"] = -1
  neu_df["polarity"] = 0
  return pd.concat([pos_df, neg_df,neu_df]).sample(frac=1).reset_index(drop=True)

train = load_dataset(train_df,info=True,txt=True)
dev = load_dataset(dev_df,info=True,txt=True)
test = load_dataset(test_random_df,info=True,txt=True)
test_fixed = load_dataset(test_fixed_df,info=True,txt=True)

train_txt = load_dataset(train_df,info=False,txt=True)
dev_txt = load_dataset(dev_df,info=False,txt=True)
test_txt = load_dataset(test_random_df,info=False,txt=True)
test_fixed_txt = load_dataset(test_fixed_df,info=False,txt=True)


train_label = load_dataset(train_df,info=True,txt=False)
dev_label = load_dataset(dev_df,info=True,txt=False)
test_label = load_dataset(test_random_df,info=True,txt=False)
test_fixed_label = load_dataset(test_fixed_df,info=True,txt=False)

print('Train length: %d, dev lenght: %d, test length: %d, fixed test length: %d'
      %(len(train),len(dev),len(test),len(test_fixed)))

number of one-paragraph docs: 252


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:149: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:151: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-v

number of one-paragraph docs: 45
document 277 has 10 paragraph labels and 11 paragraphs
number of one-paragraph docs: 52
number of one-paragraph docs: 114
number of one-paragraph docs: 252
number of one-paragraph docs: 45
document 277 has 10 paragraph labels and 11 paragraphs
number of one-paragraph docs: 52
number of one-paragraph docs: 114
number of one-paragraph docs: 252
number of one-paragraph docs: 45
document 277 has 10 paragraph labels and 11 paragraphs
number of one-paragraph docs: 52
number of one-paragraph docs: 114
Train length: 3355, dev lenght: 578, test length: 578, fixed test length: 827


In [0]:
DATA_COLUMN = 'sentence'
LABEL_COLUMN = 'polarity'
label_list = [-1, 0, 1]


In [0]:
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

dev_InputExamples = dev.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)


test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples_fixed = test_fixed.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)



train_info_InputExamples = train_label.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

dev_info_InputExamples = dev_label.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)


test_info_InputExamples = test_label.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_inf_InputExamples_fixed = test_fixed_label.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)


train_txt_InputExamples = train_txt.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

dev_txt_InputExamples = dev_txt.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)


test_txt_InputExamples = test_txt.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_txt_InputExamples_fixed = test_fixed_txt.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

## These two lines should be activated if ** is not activated
# num_train_steps = int(len(train_InputExamples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
# num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# # Setup TPU related config
# tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
# NUM_TPU_CORES = 8
# # ITERATIONS_PER_LOOP = 1000 # I don't know what it is doing just decrease it to smaller value
# ITERATIONS_PER_LOOP = int(len(train_InputExamples) / TRAIN_BATCH_SIZE) ## set as the number of iterations in each epoch 

# def get_run_config(output_dir):
#   return tf.contrib.tpu.RunConfig(
#     cluster=tpu_cluster_resolver,
#     model_dir=output_dir,
#     save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
#     tpu_config=tf.contrib.tpu.TPUConfig(
#         iterations_per_loop=ITERATIONS_PER_LOOP,
#         num_shards=NUM_TPU_CORES,
#         per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))



In [0]:
# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Functions and classes related to optimization (weight updates)."""

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import re
import tensorflow as tf


def create_optimizer(loss, init_lr, num_train_steps, num_warmup_steps, use_tpu):
  """Creates an optimizer training op."""
  global_step = tf.train.get_or_create_global_step()

  learning_rate = tf.constant(value=init_lr, shape=[], dtype=tf.float32)

  # Implements linear decay of the learning rate.
  learning_rate = tf.train.polynomial_decay(
      learning_rate,
      global_step,
      num_train_steps,
      end_learning_rate=0.0,
      power=1.0,
      cycle=False)

  # Implements linear warmup. I.e., if global_step < num_warmup_steps, the
  # learning rate will be `global_step/num_warmup_steps * init_lr`.
  if num_warmup_steps:
    global_steps_int = tf.cast(global_step, tf.int32)
    warmup_steps_int = tf.constant(num_warmup_steps, dtype=tf.int32)

    global_steps_float = tf.cast(global_steps_int, tf.float32)
    warmup_steps_float = tf.cast(warmup_steps_int, tf.float32)

    warmup_percent_done = global_steps_float / warmup_steps_float
    warmup_learning_rate = init_lr * warmup_percent_done

    is_warmup = tf.cast(global_steps_int < warmup_steps_int, tf.float32)
    learning_rate = (
        (1.0 - is_warmup) * learning_rate + is_warmup * warmup_learning_rate)

  # It is recommended that you use this optimizer for fine tuning, since this
  # is how the model was trained (note that the Adam m/v variables are NOT
  # loaded from init_checkpoint.)
  optimizer = AdamWeightDecayOptimizer(
      learning_rate=learning_rate,
      weight_decay_rate=0.01,
      beta_1=0.9,
      beta_2=0.999,
      epsilon=1e-6,
      exclude_from_weight_decay=["LayerNorm", "layer_norm", "bias"])

  if use_tpu:
    optimizer = tf.contrib.tpu.CrossShardOptimizer(optimizer)

  tvars = tf.trainable_variables()
  grads = tf.gradients(loss, tvars)

  # This is how the model was pre-trained.
  (grads, _) = tf.clip_by_global_norm(grads, clip_norm=1.0)

  train_op = optimizer.apply_gradients(
      zip(grads, tvars), global_step=global_step)

  # Normally the global step update is done inside of `apply_gradients`.
  # However, `AdamWeightDecayOptimizer` doesn't do this. But if you use
  # a different optimizer, you should probably take this line out.
  new_global_step = global_step + 1
  train_op = tf.group(train_op, [global_step.assign(new_global_step)])
  return train_op


class AdamWeightDecayOptimizer(tf.train.Optimizer):
  """A basic Adam optimizer that includes "correct" L2 weight decay."""

  def __init__(self,
               learning_rate,
               weight_decay_rate=0.0,
               beta_1=0.9,
               beta_2=0.999,
               epsilon=1e-6,
               exclude_from_weight_decay=None,
               name="AdamWeightDecayOptimizer"):
    """Constructs a AdamWeightDecayOptimizer."""
    super(AdamWeightDecayOptimizer, self).__init__(False, name)

    self.learning_rate = learning_rate
    self.weight_decay_rate = weight_decay_rate
    self.beta_1 = beta_1
    self.beta_2 = beta_2
    self.epsilon = epsilon
    self.exclude_from_weight_decay = exclude_from_weight_decay

  def apply_gradients(self, grads_and_vars, global_step=None, name=None):
    """See base class."""
    assignments = []
    for (grad, param) in grads_and_vars:
      if grad is None or param is None:
        continue

      param_name = self._get_variable_name(param.name)

      m = tf.get_variable(
          name=param_name + "/adam_m",
          shape=param.shape.as_list(),
          dtype=tf.float32,
          trainable=False,
          initializer=tf.zeros_initializer())
      v = tf.get_variable(
          name=param_name + "/adam_v",
          shape=param.shape.as_list(),
          dtype=tf.float32,
          trainable=False,
          initializer=tf.zeros_initializer())

      # Standard Adam update.
      next_m = (
          tf.multiply(self.beta_1, m) + tf.multiply(1.0 - self.beta_1, grad))
      next_v = (
          tf.multiply(self.beta_2, v) + tf.multiply(1.0 - self.beta_2,
                                                    tf.square(grad)))

      update = next_m / (tf.sqrt(next_v) + self.epsilon)

      # Just adding the square of the weights to the loss function is *not*
      # the correct way of using L2 regularization/weight decay with Adam,
      # since that will interact with the m and v parameters in strange ways.
      #
      # Instead we want ot decay the weights in a manner that doesn't interact
      # with the m/v parameters. This is equivalent to adding the square
      # of the weights to the loss with plain (non-momentum) SGD.
      if self._do_use_weight_decay(param_name):
        update += self.weight_decay_rate * param

      update_with_lr = self.learning_rate * update

      next_param = param - update_with_lr

      assignments.extend(
          [param.assign(next_param),
           m.assign(next_m),
           v.assign(next_v)])
    return tf.group(*assignments, name=name)

  def _do_use_weight_decay(self, param_name):
    """Whether to use L2 weight decay for `param_name`."""
    if not self.weight_decay_rate:
      return False
    if self.exclude_from_weight_decay:
      for r in self.exclude_from_weight_decay:
        if re.search(r, param_name) is not None:
          return False
    return True

  def _get_variable_name(self, param_name):
    """Get the variable name from the tensor name."""
    m = re.match("^(.*):\\d+$", param_name)
    if m is not None:
      param_name = m.group(1)
    return param_name


In [0]:
# weight_loss = 
# neg_w = 1./len(dev_par[dev_par['label']==0])
# pos_w = 1./len(dev_par[dev_par['label']==1])
# class_weights_arr = [neg_w/(neg_w+pos_w),pos_w/(neg_w+pos_w)]
def create_model(is_training, input_ids, input_mask, segment_ids, labels,
                 num_labels, bert_hub_module_handle):
  """Creates a classification model."""
  tags = set()
  if is_training:
    tags.add("train")
  bert_module = hub.Module(bert_hub_module_handle, tags=tags, trainable=trainable)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # In the demo, we are doing a simple classification task on the entire
  # segment.
  #
  # If you want to use the token-level output, use
  # bert_outputs["sequence_output"] instead.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):
    if is_training:
      # I.e., 0.1 dropout
      output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    probabilities = tf.nn.softmax(logits, axis=-1)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)# * weight_loss)

    return (loss, per_example_loss, logits, probabilities)


def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps, use_tpu, bert_hub_module_handle):
  """Returns `model_fn` closure for TPUEstimator."""

  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    tf.logging.info("*** Features ***")
    for name in sorted(features.keys()):
      tf.logging.info("  name = %s, shape = %s" % (name, features[name].shape))

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_training = (mode == tf.estimator.ModeKeys.TRAIN)

    (total_loss, per_example_loss, logits, probabilities) = create_model(
        is_training, input_ids, input_mask, segment_ids, label_ids, num_labels,
        bert_hub_module_handle)

    output_spec = None
    if mode == tf.estimator.ModeKeys.TRAIN:
      train_op =create_optimizer(
          total_loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu)

      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          train_op=train_op)
    elif mode == tf.estimator.ModeKeys.EVAL:

      def metric_fn(per_example_loss, label_ids, logits):
        predictions = tf.argmax(logits, axis=-1, output_type=tf.int32)
        accuracy = tf.metrics.accuracy(label_ids, predictions)
        loss = tf.metrics.mean(per_example_loss)
        return {
            "eval_accuracy": accuracy,
            "eval_loss": loss,
        }

      eval_metrics = (metric_fn, [per_example_loss, label_ids, logits])
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode,
          loss=total_loss,
          eval_metrics=eval_metrics)
    elif mode == tf.estimator.ModeKeys.PREDICT:
      output_spec = tf.contrib.tpu.TPUEstimatorSpec(
          mode=mode, predictions={"probabilities": probabilities})
    else:
      raise ValueError(
          "Only TRAIN, EVAL and PREDICT modes are supported: %s" % (mode))

    return output_spec

  return model_fn

In [11]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

# Force TF Hub writes to the GS bucket we provide.
# num_train_steps = int(len(train_InputExamples_par) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_train_steps = int(len(train_InputExamples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
# ITERATIONS_PER_LOOP = 1000 # I don't know what it is doing just decrease it to smaller value
# ITERATIONS_PER_LOOP = int(len(train_InputExamples_par) / TRAIN_BATCH_SIZE) ## set as the number of iterations in each epoch 
ITERATIONS_PER_LOOP = int(len(train_InputExamples) / TRAIN_BATCH_SIZE) ## set as the number of iterations in each epoch 

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    keep_checkpoint_max=keep_checkpoint_max,
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

### Activate it if ** part is not activated 
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)



W1114 06:09:58.688245 140638232864640 estimator.py:1994] Estimator's model_fn (<function model_fn at 0x7fe8a00e15f0>) includes params argument, but params are not passed to Estimator.


In [0]:
tf.logging.set_verbosity(tf.logging.FATAL)
train_features = run_classifier.convert_examples_to_features(
      train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
eval_features = run_classifier.convert_examples_to_features(
      dev_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = run_classifier.convert_examples_to_features(
      test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

test_features_fixed = run_classifier.convert_examples_to_features(
      test_InputExamples_fixed, label_list, MAX_SEQ_LENGTH, tokenizer)



train_info_features = run_classifier.convert_examples_to_features(
      train_info_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
eval_info_features = run_classifier.convert_examples_to_features(
      dev_info_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_info_features = run_classifier.convert_examples_to_features(
      test_info_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

test_info_features_fixed = run_classifier.convert_examples_to_features(
      test_inf_InputExamples_fixed, label_list, MAX_SEQ_LENGTH, tokenizer)




train_txt_features = run_classifier.convert_examples_to_features(
      train_txt_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
eval_txt_features = run_classifier.convert_examples_to_features(
      dev_txt_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_txt_features = run_classifier.convert_examples_to_features(
      test_txt_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

test_txt_features_fixed = run_classifier.convert_examples_to_features(
      test_txt_InputExamples_fixed, label_list, MAX_SEQ_LENGTH, tokenizer)

In [0]:
# Train the model
# tf.logging.set_verbosity(tf.logging.INFO) #DEBUG,ERROR,FATAL,INFO,WARN
def model_train(estimator,train_features):
  # We'll set sequences to be at most 128 tokens long.
#   train_features = run_classifier.convert_examples_to_features(
#       train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_InputExamples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  md = estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))
  return md



#Evaluation and Prediction 

In [0]:
def model_eval(estimator,eval_examples=None,eval_features=None):
  # Eval the model.
#   eval_examples = dev_InputExamples#processor.get_dev_examples(TASK_DATA_DIR)
#   eval_features = run_classifier.convert_examples_to_features(
#       eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))
      
  return result

In [15]:
import numpy as np
from sklearn import metrics
labels = ["Negative","Neutral", "Positive"]
def model_predict(estimator,prediction_examples,input_features,checkpoint_path=None):
  # Make predictions on a subset of eval examples
#   prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]
#   input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  if checkpoint_path: 
    predictions = estimator.predict(predict_input_fn,checkpoint_path=checkpoint_path)
  else:
    predictions = estimator.predict(predict_input_fn)
  predicted_values =  [prediction['probabilities'] for prediction in predictions]
  print('prediction length: %d'%len(predicted_values))
  print('input length: %d'%len(prediction_examples))
  return predicted_values

model_train(estimator_from_tfhub,train_features=train_features)

***** Started training at 2019-11-14 06:11:23.946257 *****
  Num examples = 3355
  Batch size = 32
***** Finished training at 2019-11-14 06:11:24.643495 *****


In [0]:
# tf.logging.set_verbosity(tf.logging.FATAL)
# pd = model_predict(estimator_from_tfhub,train_InputExamples,train_features,checkpoint_path=OUTPUT_DIR+'/model.ckpt-230')
# labels_val = []
# true_label = list(train['sentiment'])
# for item in pd:
#     labels_val.append(labels[np.argmax(item)])
# print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
# print(metrics.classification_report(y_pred=labels_val,y_true = true_label))




# for ckpt in range(138,275,46):
#   print('evaluating ckpt %d'%ckpt)
#   pd = model_predict(estimator_from_tfhub,dev_InputExamples[:200],eval_features[:200],checkpoint_path=OUTPUT_DIR+'/model.ckpt-%d'%ckpt)
#   labels_val = []
#   true_label = list(dev['sentiment'])[:200]
#   for item in pd:
#       labels_val.append(labels[np.argmax(item[1])])
#   print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
#   print(metrics.classification_report(y_pred=labels_val,y_true = true_label))
  
  
#   pd = model_predict(estimator_from_tfhub,train_InputExamples,train_features,checkpoint_path=OUTPUT_DIR+'/model.ckpt-%d'%ckpt)
#   labels_val = []
#   true_label = list(train['sentiment'])
#   for item in pd:
#       labels_val.append(labels[np.argmax(item[1])])
#   print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
#   print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

In [17]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'
# tf.logging.set_verbosity(tf.logging.FATAL)
for ckpt in range(0,209,104):  
  print('evaluating on checkpoint: %d'%ckpt)
  pd = model_predict(estimator_from_tfhub,dev_InputExamples,eval_features,checkpoint_path=OUTPUT_DIR+'/model.ckpt-%d'%ckpt)
  labels_val = []
  true_label = list(dev['sentiment'])
  for item in pd:
        labels_val.append(labels[np.argmax(item)])
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

evaluating on checkpoint: 0


E1114 06:11:31.121153 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/input_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:11:31.123198 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/input_mask) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:11:31.125782 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/segment_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:11:31.128063 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/mlm_positions) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:11:31.136178 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/bert/embeddings/word_embeddings) is not supported on the TPU. Execution will fail if this op is used in the grap

prediction length: 578
input length: 578
[[  0   0  58]
 [  6   2 208]
 [  1   3 300]]
              precision    recall  f1-score   support

    Negative       0.00      0.00      0.00        58
     Neutral       0.40      0.01      0.02       216
    Positive       0.53      0.99      0.69       304

   micro avg       0.52      0.52      0.52       578
   macro avg       0.31      0.33      0.24       578
weighted avg       0.43      0.52      0.37       578

evaluating on checkpoint: 104


E1114 06:12:04.715008 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/input_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:12:04.717053 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/input_mask) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:12:04.721724 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/segment_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:12:04.723975 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/mlm_positions) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:12:04.734770 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/bert/embeddings/word_embeddings) is not supported on the TPU. Execution will fail if this op is used in the grap

prediction length: 578
input length: 578
[[ 39  19   0]
 [ 13 165  38]
 [  0  49 255]]
              precision    recall  f1-score   support

    Negative       0.75      0.67      0.71        58
     Neutral       0.71      0.76      0.73       216
    Positive       0.87      0.84      0.85       304

   micro avg       0.79      0.79      0.79       578
   macro avg       0.78      0.76      0.77       578
weighted avg       0.80      0.79      0.80       578

evaluating on checkpoint: 208


E1114 06:12:41.043725 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/input_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:12:41.045552 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/input_mask) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:12:41.048844 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/segment_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:12:41.050825 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/mlm_positions) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:12:41.059603 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/bert/embeddings/word_embeddings) is not supported on the TPU. Execution will fail if this op is used in the grap

prediction length: 578
input length: 578
[[ 39  18   1]
 [ 14 147  55]
 [  1  21 282]]
              precision    recall  f1-score   support

    Negative       0.72      0.67      0.70        58
     Neutral       0.79      0.68      0.73       216
    Positive       0.83      0.93      0.88       304

   micro avg       0.81      0.81      0.81       578
   macro avg       0.78      0.76      0.77       578
weighted avg       0.81      0.81      0.81       578



In [18]:
tf.logging.set_verbosity(tf.logging.ERROR)
pd = model_predict(estimator_from_tfhub,dev_InputExamples,eval_features,checkpoint_path=OUTPUT_DIR+'/model.ckpt-104')
labels_val = []
true_label = list(dev['sentiment'])
for item in pd:
    labels_val.append(labels[np.argmax(item)])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

pd = model_predict(estimator_from_tfhub,test_InputExamples,test_features,checkpoint_path=OUTPUT_DIR+'/model.ckpt-104')
labels_val = []
true_label = list(test['sentiment'])
for item in pd:
    labels_val.append(labels[np.argmax(item)])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

pd = model_predict(estimator_from_tfhub,test_InputExamples_fixed,test_features_fixed,checkpoint_path=OUTPUT_DIR+'/model.ckpt-104')
labels_val = []
true_label = list(test_fixed['sentiment'])
for item in pd:
    labels_val.append(labels[np.argmax(item)])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

E1114 06:13:20.042604 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/input_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:13:20.044639 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/input_mask) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:13:20.047796 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/segment_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:13:20.050333 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/mlm_positions) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:13:20.058831 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/bert/embeddings/word_embeddings) is not supported on the TPU. Execution will fail if this op is used in the grap

prediction length: 578
input length: 578
[[ 39  19   0]
 [ 13 165  38]
 [  0  49 255]]
              precision    recall  f1-score   support

    Negative       0.75      0.67      0.71        58
     Neutral       0.71      0.76      0.73       216
    Positive       0.87      0.84      0.85       304

   micro avg       0.79      0.79      0.79       578
   macro avg       0.78      0.76      0.77       578
weighted avg       0.80      0.79      0.80       578



E1114 06:13:54.771832 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/input_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:13:54.774027 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/input_mask) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:13:54.779084 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/segment_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:13:54.781791 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/mlm_positions) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:13:54.791081 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/bert/embeddings/word_embeddings) is not supported on the TPU. Execution will fail if this op is used in the grap

prediction length: 578
input length: 578
[[ 59  13   1]
 [ 11 170  32]
 [  1  41 250]]
              precision    recall  f1-score   support

    Negative       0.83      0.81      0.82        73
     Neutral       0.76      0.80      0.78       213
    Positive       0.88      0.86      0.87       292

   micro avg       0.83      0.83      0.83       578
   macro avg       0.82      0.82      0.82       578
weighted avg       0.83      0.83      0.83       578



E1114 06:14:31.624454 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/input_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:14:31.626244 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/input_mask) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:14:31.627700 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/segment_ids) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:14:31.629004 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/mlm_positions) is not supported on the TPU. Execution will fail if this op is used in the graph. 
E1114 06:14:31.636100 140638232864640 tpu.py:425] Operation of type Placeholder (module_apply_tokens/bert/embeddings/word_embeddings) is not supported on the TPU. Execution will fail if this op is used in the grap

prediction length: 827
input length: 827
[[107  32   0]
 [ 22 253  45]
 [  3  77 288]]
              precision    recall  f1-score   support

    Negative       0.81      0.77      0.79       139
     Neutral       0.70      0.79      0.74       320
    Positive       0.86      0.78      0.82       368

   micro avg       0.78      0.78      0.78       827
   macro avg       0.79      0.78      0.78       827
weighted avg       0.79      0.78      0.79       827



In [0]:
for ckpt in range(92,277,46):  
  print('evaluating on checkpoint: %d'%ckpt)
  pd = model_predict(estimator_from_tfhub,dev_info_InputExamples,eval_info_features,checkpoint_path=OUTPUT_DIR+'/model.ckpt-%d'%ckpt)
  labels_val = []
  true_label = list(dev_label['sentiment'])
  for item in pd:
        labels_val.append(labels[np.argmax(item)])
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

evaluating on checkpoint: 92
prediction length: 283
input length: 283
[[ 16   6   1]
 [  3  71  23]
 [  0   9 154]]
              precision    recall  f1-score   support

    Negative       0.84      0.70      0.76        23
     Neutral       0.83      0.73      0.78        97
    Positive       0.87      0.94      0.90       163

   micro avg       0.85      0.85      0.85       283
   macro avg       0.84      0.79      0.81       283
weighted avg       0.85      0.85      0.85       283

evaluating on checkpoint: 138
prediction length: 283
input length: 283
[[ 19   4   0]
 [  5  68  24]
 [  0   9 154]]
              precision    recall  f1-score   support

    Negative       0.79      0.83      0.81        23
     Neutral       0.84      0.70      0.76        97
    Positive       0.87      0.94      0.90       163

   micro avg       0.85      0.85      0.85       283
   macro avg       0.83      0.82      0.83       283
weighted avg       0.85      0.85      0.85       283

evalu

In [0]:
import pandas as pd

def get_prediction_set(train_prediction,lcolumn_label='prediction'):
  import pandas as pd
  texts = []
  labels = []
  pred_labels = []
  new_train = pd.DataFrame([])
  train_labels = pd.DataFrame([])
  for doc_id in train_prediction['doc_id'].unique():
    paragraphs = train_prediction[train_prediction['doc_id']==doc_id]

    docs = []
    just_labels = []
    for par,label in zip(paragraphs['sentence'],paragraphs[lcolumn_label]):
      par = par.replace('[TGT]','tgt')
      docs.append(label+' '+par)
      just_labels.append(label)
#         docs.append(par)
#         docs.append(label)
    texts.append('\n'.join(docs))
    pred_labels.append('\n'.join(just_labels))
    labels.append(list(paragraphs['doc_label'])[0])
  new_train['sentence'] = pd.Series(texts)
  new_train['sentiment'] = pd.Series(labels)
  
  train_labels['sentence'] = pd.Series(pred_labels)
  train_labels['sentiment'] = pd.Series(labels)
  
  pos_df = new_train[new_train['sentiment'] == 'Positive']
  neg_df = new_train[new_train['sentiment'] == 'Negative']
  neu_df = new_train[new_train['sentiment'] == 'Neutral']
  pos_df["polarity"] = 1
  neg_df["polarity"] = -1
  neu_df["polarity"] = 0
  train_pred =  pd.concat([pos_df, neg_df,neu_df]).sample(frac=1).reset_index(drop=True)
  
  pos_df = train_labels[train_labels['sentiment'] == 'Positive']
  neg_df = train_labels[train_labels['sentiment'] == 'Negative']
  neu_df = train_labels[train_labels['sentiment'] == 'Neutral']
  pos_df["polarity"] = 1
  neg_df["polarity"] = -1
  neu_df["polarity"] = 0
  train_labels =  pd.concat([pos_df, neg_df,neu_df]).sample(frac=1).reset_index(drop=True)
  
  train_pred_InputExamples = train_pred.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)
  train_pred_features = run_classifier.convert_examples_to_features(train_pred_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
  
  
  train_labels_InputExamples = train_labels.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)
  train_labels_features = run_classifier.convert_examples_to_features(train_labels_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
 
  
  return (train_pred,train_pred_InputExamples,train_pred_features,
         train_labels,train_labels_InputExamples,train_labels_features)

print('------------Train------------')

train_prediction = pd.read_csv('train_prediction_v2.csv')
train_pred,train_pred_InputExamples,train_pred_features, \
train_label_pred,train_label_pred_InputExamples,train_label_pred_features= \
 get_prediction_set(train_prediction,'sentiment')

print('------------Dev------------')
dev_prediction = pd.read_csv('dev_prediction_v2.csv')
dev_pred,dev_pred_InputExamples,dev_pred_features,\
dev_label_pred,dev_label_pred_InputExamples,dev_label_pred_features \
= get_prediction_set(dev_prediction,'prediction')

print('------------Random Test------------')

test_prediction = pd.read_csv('test_prediction_v2.csv')
test_pred,test_pred_InputExamples,test_pred_features, \
test_label_pred,test_label_pred_InputExamples,test_label_pred_features \
= get_prediction_set(test_prediction,'prediction')

print('------------Fixed Test------------')

test_fixed_prediction = pd.read_csv('test_fixed_prediction_v2.csv')
test_fixed_pred,test_fixed_pred_InputExamples,test_fixed_pred_features,\
test_fixed_label_pred,test_fixed_label_pred_InputExamples,test_fixed_label_pred_features \
= get_prediction_set(test_fixed_prediction,'prediction')

## These two lines should be activated if ** is not activated
num_train_steps = int(len(train_pred_InputExamples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
# ITERATIONS_PER_LOOP = 1000 # I don't know what it is doing just decrease it to smaller value
ITERATIONS_PER_LOOP = int(len(train_pred_InputExamples) / TRAIN_BATCH_SIZE) ## set as the number of iterations in each epoch 

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))




# test_InputExamples_fixed = test_fixed.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
#                                                                    text_a = x[DATA_COLUMN], 
#                                                                    text_b = None, 
#                                                                    label = x[LABEL_COLUMN]), axis = 1)

------------Train------------


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

------------Dev------------
------------Random Test------------
------------Fixed Test------------


In [0]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR
### Activate it if ** part is not activated 
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)

In [0]:
tf.logging.set_verbosity(tf.logging.FATAL)
def model_train_prediction(estimator):
  # We'll set sequences to be at most 128 tokens long.
#   train_features = run_classifier.convert_examples_to_features(
#       train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_pred_InputExamples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_pred_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  md = estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))
  return md



model_train_prediction(estimator_from_tfhub)

***** Started training at 2019-05-18 16:20:33.314466 *****
  Num examples = 1207
  Batch size = 32


/usr/local/lib/python2.7/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


***** Finished training at 2019-05-18 16:28:14.882797 *****


In [0]:
tf.logging.set_verbosity(tf.logging.INFO)
pd = model_predict(estimator_from_tfhub,train_pred_InputExamples,train_pred_features,checkpoint_path=OUTPUT_DIR+'/Best_model_True_label_and_text/model.ckpt-92')
labels_val = []
true_label = list(train_pred['sentiment'])
for item in pd:
    labels_val.append(labels[np.argmax(item)])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

INFO:tensorflow:Calling model_fn.


I0521 04:57:00.270524 140699154761600 estimator.py:1111] Calling model_fn.


INFO:tensorflow:*** Features ***


I0521 04:57:04.531338 140699154761600 <ipython-input-10-5033312e856d>:58] *** Features ***


INFO:tensorflow:  name = input_ids, shape = (1, 512)


...

INFO:tensorflow:prediction_loop marked as finished


I0521 04:57:42.987447 140699154761600 error_handling.py:93] prediction_loop marked as finished


INFO:tensorflow:prediction_loop marked as finished


I0521 04:57:42.990092 140699154761600 error_handling.py:93] prediction_loop marked as finished


prediction length: 1501
input length: 1501
[[117  27   1]
 [  9 299 135]
 [  0  34 879]]
              precision    recall  f1-score   support

    Negative       0.93      0.81      0.86       145
     Neutral       0.83      0.67      0.74       443
    Positive       0.87      0.96      0.91       913

   micro avg       0.86      0.86      0.86      1501
   macro avg       0.88      0.81      0.84      1501
weighted avg       0.86      0.86      0.86      1501



In [0]:
tf.logging.set_verbosity(tf.logging.FATAL)
pd = model_predict(estimator_from_tfhub,dev_pred_InputExamples,dev_pred_features,checkpoint_path=OUTPUT_DIR+'/Best_model_True_label_and_text/model.ckpt-92')
labels_val = []
true_label = list(dev_pred['sentiment'])
for item in pd:
    labels_val.append(labels[np.argmax(item)])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

prediction length: 284
input length: 284
[[  3   8  12]
 [  1  35  61]
 [  0  23 141]]
              precision    recall  f1-score   support

    Negative       0.75      0.13      0.22        23
     Neutral       0.53      0.36      0.43        97
    Positive       0.66      0.86      0.75       164

   micro avg       0.63      0.63      0.63       284
   macro avg       0.65      0.45      0.47       284
weighted avg       0.62      0.63      0.60       284



In [0]:
# tf.logging.set_verbosity(tf.logging.FATAL)
# pd = model_predict(estimator_from_tfhub,test_pred_InputExamples,test_pred_features,checkpoint_path=OUTPUT_DIR+'/Best_model_True_label_and_text/model.ckpt-92')
true_label = list(test_pred['sentiment'])
labels_val = []
for item in pd:
    labels_val.append(labels[np.argmax(item)])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

[[  2  10  16]
 [  0  33  54]
 [  4  21 144]]
              precision    recall  f1-score   support

    Negative       0.33      0.07      0.12        28
     Neutral       0.52      0.38      0.44        87
    Positive       0.67      0.85      0.75       169

   micro avg       0.63      0.63      0.63       284
   macro avg       0.51      0.43      0.44       284
weighted avg       0.59      0.63      0.59       284



In [0]:
tf.logging.set_verbosity(tf.logging.FATAL)
pd = model_predict(estimator_from_tfhub,test_fixed_pred_InputExamples,test_fixed_pred_features,checkpoint_path=OUTPUT_DIR+'/Best_model_True_label_and_text/model.ckpt-92')
labels_val = []
true_label = list(test_fixed_pred['sentiment'])
for item in pd:
    labels_val.append(labels[np.argmax(item)])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

prediction length: 339
input length: 339
[[  8  18  24]
 [  9  34  73]
 [  2  30 141]]
              precision    recall  f1-score   support

    Negative       0.42      0.16      0.23        50
     Neutral       0.41      0.29      0.34       116
    Positive       0.59      0.82      0.69       173

   micro avg       0.54      0.54      0.54       339
   macro avg       0.48      0.42      0.42       339
weighted avg       0.51      0.54      0.50       339



In [0]:
#### train on predictions:
tf.logging.set_verbosity(tf.logging.FATAL)
def model_train_prediction(estimator):
  # We'll set sequences to be at most 128 tokens long.
#   train_features = run_classifier.convert_examples_to_features(
#       train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_pred_InputExamples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_pred_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  md = estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))
  return md



model_train_prediction(estimator_from_tfhub)

***** Started training at 2019-05-21 05:17:37.776008 *****
  Num examples = 1501
  Batch size = 32
***** Finished training at 2019-05-21 05:23:26.906435 *****


In [0]:
tf.logging.set_verbosity(tf.logging.FATAL)
# 281: 47
for ckpt in range(138,277,46):
  pd = model_predict(estimator_from_tfhub,dev_pred_InputExamples,dev_pred_features,checkpoint_path=OUTPUT_DIR+'/model.ckpt-%d'%ckpt)
  labels_val = []
  true_label = list(dev_pred['sentiment'])
  for item in pd:
      labels_val.append(labels[np.argmax(item)])
  print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
  print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

prediction length: 284
input length: 284
[[  3   9  11]
 [  1  34  62]
 [  0  19 145]]
              precision    recall  f1-score   support

    Negative       0.75      0.13      0.22        23
     Neutral       0.55      0.35      0.43        97
    Positive       0.67      0.88      0.76       164

   micro avg       0.64      0.64      0.64       284
   macro avg       0.65      0.46      0.47       284
weighted avg       0.63      0.64      0.60       284

prediction length: 284
input length: 284
[[  3   9  11]
 [  1  43  53]
 [  0  29 135]]
              precision    recall  f1-score   support

    Negative       0.75      0.13      0.22        23
     Neutral       0.53      0.44      0.48        97
    Positive       0.68      0.82      0.74       164

   micro avg       0.64      0.64      0.64       284
   macro avg       0.65      0.47      0.48       284
weighted avg       0.63      0.64      0.61       284

prediction length: 284
input length: 284
[[  3   8  12]
 [  1  4

In [0]:
tf.logging.set_verbosity(tf.logging.FATAL)
pd = model_predict(estimator_from_tfhub,test_fixed_pred_InputExamples,test_fixed_pred_features,checkpoint_path=OUTPUT_DIR+'/model.ckpt-230')
labels_val = []
true_label = list(test_fixed_pred['sentiment'])
for item in pd:
    labels_val.append(labels[np.argmax(item)])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

prediction length: 339
input length: 339
[[  8  18  24]
 [  9  39  68]
 [  2  37 134]]
              precision    recall  f1-score   support

    Negative       0.42      0.16      0.23        50
     Neutral       0.41      0.34      0.37       116
    Positive       0.59      0.77      0.67       173

   micro avg       0.53      0.53      0.53       339
   macro avg       0.48      0.42      0.42       339
weighted avg       0.51      0.53      0.50       339



In [0]:
tf.logging.set_verbosity(tf.logging.FATAL)
pd = model_predict(estimator_from_tfhub,test_pred_InputExamples,test_pred_features,checkpoint_path=OUTPUT_DIR+'/model.ckpt-230')
labels_val = []
true_label = list(test_pred['sentiment'])
for item in pd:
    labels_val.append(labels[np.argmax(item)])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

prediction length: 284
input length: 284
[[  2  10  16]
 [  0  35  52]
 [  2  30 137]]
              precision    recall  f1-score   support

    Negative       0.50      0.07      0.12        28
     Neutral       0.47      0.40      0.43        87
    Positive       0.67      0.81      0.73       169

   micro avg       0.61      0.61      0.61       284
   macro avg       0.54      0.43      0.43       284
weighted avg       0.59      0.61      0.58       284



In [0]:
##### train on JUST True Labels and predict with predicted labels
print('DEV')
pd = model_predict(estimator_from_tfhub,dev_label_pred_InputExamples,dev_label_pred_features,checkpoint_path=OUTPUT_DIR+'/Best_Model_True_Label/model.ckpt-138')
labels_val = []
true_label = list(dev_label_pred['sentiment'])
for item in pd:
      labels_val.append(labels[np.argmax(item)])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))
  
print('RANDOM TEST')
pd = model_predict(estimator_from_tfhub,test_label_pred_InputExamples,test_label_pred_features,checkpoint_path=OUTPUT_DIR+'/Best_Model_True_Label/model.ckpt-138')
labels_val = []
true_label = list(test_label_pred['sentiment'])
for item in pd:
    labels_val.append(labels[np.argmax(item)])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))



print('FIXED TEST')
tf.logging.set_verbosity(tf.logging.FATAL)
pd = model_predict(estimator_from_tfhub,test_fixed_label_pred_InputExamples,test_fixed_label_pred_features,checkpoint_path=OUTPUT_DIR+'/Best_Model_True_Label/model.ckpt-138')
labels_val = []
true_label = list(test_fixed_label_pred['sentiment'])
for item in pd:
    labels_val.append(labels[np.argmax(item)])
print(metrics.confusion_matrix(y_pred=labels_val,y_true=true_label))
print(metrics.classification_report(y_pred=labels_val,y_true = true_label))

DEV
prediction length: 284
input length: 284
[[  5   9   9]
 [  2  41  54]
 [  1  28 135]]
              precision    recall  f1-score   support

    Negative       0.62      0.22      0.32        23
     Neutral       0.53      0.42      0.47        97
    Positive       0.68      0.82      0.75       164

   micro avg       0.64      0.64      0.64       284
   macro avg       0.61      0.49      0.51       284
weighted avg       0.62      0.64      0.62       284

RANDOM TEST
prediction length: 284
input length: 284
[[  3  10  15]
 [  2  38  47]
 [  4  31 134]]
              precision    recall  f1-score   support

    Negative       0.33      0.11      0.16        28
     Neutral       0.48      0.44      0.46        87
    Positive       0.68      0.79      0.73       169

   micro avg       0.62      0.62      0.62       284
   macro avg       0.50      0.45      0.45       284
weighted avg       0.59      0.62      0.59       284

FIXED TEST
prediction length: 339
input length: 